# Actualización aplicación Color APP 26-10-22

Se adicionó modulo calculo de color y delta E para colorantes 

In [4]:
## https://sparkbyexamples.com/pandas/pandas-read-sql-query-or-table/  pandas sql
#https://www.youtube.com/watch?v=HD5xacNn-5o   parte1
#https://www.youtube.com/watch?v=fqicq50Nkk0   parte2
#https://www.youtube.com/watch?v=mPty7RQ5JiU  27
#https://www.youtube.com/watch?v=y4vF4PZoBFM&list=PLjARR1053fYlEKgn0H1-x3UM9Zl4aYLxy&index=28 26
#https://www.youtube.com/watch?v=OjMDXTlVOYU
# https://www.youtube.com/watch?v=5ku3YVIY0x0
# https://www.tutorialspoint.com/python/tk_relief.htm
# https://itvoyagers.in/labels-and-buttons-with-relief-styles-in-tkinter/
#https://www.youtube.com/watch?v=heeghMrCMgY


# Importar Bibliotecas 
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkinter import ttk 
import sqlite3

import pandas as pd
import sqlalchemy 
from pandastable import Table, TableModel
from PIL import ImageTk, Image
import os
import datetime

import numpy as np
import cv2
from PIL import Image, ImageTk
from skimage.color import lab2rgb
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
# Desarrollo de la Interfaz grafica


logging = Tk()
logging.title('Color_analyzer_app')
logging.resizable(width=False, height=False)
logging.geometry("800x600+200+50")
logging.iconbitmap('Captura.ico')


img1 = Image.open('fondo_1.png')
img1 = img1.resize((760,540), Image.ANTIALIAS)
img1 = ImageTk.PhotoImage(img1)
img_label = tk.Label(logging, image = img1, relief=SUNKEN,borderwidth=8)
img_label.place(x=15,y=25)

##################################################################
#######################################################


## Crear usuario
usuario = StringVar()
password = StringVar()


### ACA ES PARA VOLVER A COLOCAR LOS BOTONES DE ID Y PASSWORD

## Crear funcuines para los botones entrar y salir 
def irse():
    logging.destroy()

def log():
    nombre = usuario.get()
    passw  = password.get()
    if nombre == "JDM" and passw == "1234":
        correcta()
    else:
        pass

def correcta():
    global insertdata
    logging.withdraw()
    insertdata = tk.Toplevel()
    insertdata.title('Module A')
    insertdata.geometry('1300x800')
    insertdata.config(bg='grey90')
    insertdata.iconbitmap('Captura.ico')

    img = Image.open('tkinter.png')
    img = img.resize((220,75), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    img_label = Label(insertdata, image = img, relief=SUNKEN,borderwidth=4)
    img_label.place(x=15,y=25)  


        # Crear variables
    miId=StringVar()
    Cod_Bulk = StringVar()
    L_max = StringVar()
    L_min = StringVar()
    A_max = StringVar()
    A_min = StringVar()
    B_max = StringVar()
    B_min = StringVar()
    L_patron = StringVar()
    A_patron = StringVar()
    B_patron = StringVar()

    # Crear conexion BD
    def conexionBD():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        #Crear tabla
        try:
            miCursor.execute('''
                CREATE TABLE IF NOT EXISTS especificacion(
                ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                CodBulk	INTEGER NOT NULL,
                L_Max FLOAT NOT NULL,
                L_Min FLOAT NOT NULL,
                A_Max FLOAT NOT NULL,
                A_Min FLOAT NOT NULL,
                B_Max FLOAT NOT NULL,
                B_Min FLOAT NOT NULL,
                L_Patron FLOAT NOT NULL,
                A_Patron FLOAT NOT NULL,
                B_Patron FLOAT NOT NULL
                );
            ''')
            # Cuadro de texto o dialogo para mostrar dicha información
            messagebox.showinfo('CONEXION', 'Base de datos creada exitosamente')
        except:
            messagebox.showinfo('CONEXION', 'Conexión exitosa con la Base de datos')

    # Función para eliminar la base de datos 
    def eliminarBBDD():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        if messagebox.askyesno(message="¿Los Datos se perderan definitivamente, Desea continuar?", title="ADVERTENCIA"):
            miCursor.execute('DROP TABLE especificacion')
        else:
            pass
        limpiarCampos()
        mostrar() 

    #Función salir aplicación 
    def salirAplicacion():
        valor=messagebox.askyesno("Salir","¿Está seguro que desea salir de la Aplicación?")
        if valor=='yes':
            root.destroy()


    # Función limpiar datos
    def limpiarCampos():
        miId.set('')
        Cod_Bulk.set('')
        L_max.set('')
        L_min.set('')
        A_max.set('')
        A_min.set('')
        B_max.set('')
        B_min.set('')
        L_patron.set('')
        A_patron.set('')
        B_patron.set('')


    # Función mensaje de la versión aplicación
    def mensaje():
        acerca='''
        application to control quality control in liquid colors
        version 1.0
        Technology Python Tkinter sqlite3
        Author: Juan David Marín, 
        email: qcojuandavidmarin@gmail.com
        '''
        messagebox.showinfo(title='INFORMACIÓN', message=acerca)

    ################################ Métodos CRUD ############

    # Metodo para crear 
    def crear():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        try:
            datos=Cod_Bulk.get(),L_max.get(),L_min.get(),A_max.get(),A_min.get(),B_max.get(),B_min.get(),L_patron.get(),A_patron.get(),B_patron.get()
            miCursor.execute('INSERT INTO especificacion VALUES(NULL,?,?,?,?,?,?,?,?,?,?)',(datos))
            miConexion.commit() # Commit para que se actualicen los cambios en la tabla
        except:
            messagebox.showwarning('ADVERTENCIA","Ocurrió un error al crear el registro, verifique conexión con BBDD')
            pass
        limpiarCampos()
        mostrar()


    ####_TENER EN CUENTA PARA SACAR LA DATA DE LA TABLA Y PROGRAMAR PERACIONES_############
    # Función mostrar los datos al crear un registro y que se vea en la lista de registros
    def mostrar():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        registros=tree.get_children() # Permite extraer todos los datos de la tabla
        for elemento in registros:
            tree.delete(elemento) # Este borrado es solo en la parte de la ventana donde colocamos las variables
        # Try y catch para llenar la tabla con los valores extraidos de la BD
        try:
            miCursor.execute('SELECT * FROM especificacion')
            for row in miCursor:
                tree.insert('',0,text=row[0],value=(row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10]))
        except:
            pass

    # Actualizar
    def actualizar():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        try:
            datos=Cod_Bulk.get(),L_max.get(),L_min.get(),A_max.get(),A_min.get(),B_max.get(),B_min.get(),L_patron.get(),A_patron.get(),B_patron.get()
            miCursor.execute("UPDATE especificacion SET CodBulk=?, L_Max=?, L_Min=?, A_Max=?, A_Min=?, B_Max=?, B_Min=?, L_Patron=?, A_Patron=?, B_Patron=?  WHERE ID="+miId.get(), (datos))
            miConexion.commit()
        except:
            messagebox.showwarning('ADVERTENCIA","Ocurrió un error al actualizar el registro')
            pass    
        limpiarCampos()
        mostrar() 



    # Funcion para borrar
    def borrar():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        try:
            if messagebox.askyesno(message="¿Realmente desea eliminar el registro?", title="ADVERTENCIA"):
                miCursor.execute('DELETE FROM especificacion WHERE ID='+miId.get()) # El mas (+) sirve para concatenar
                miConexion.commit()
        except:
            messagebox.showwarning("ADVERTENCIA","Ocurrió un error al tratar de eliminar el registro")
            pass
        limpiarCampos()
        mostrar() 

##################################################################
#######################################################
    ####________________________________________________________############
    # Crear la tabla 
    tree=ttk.Treeview(insertdata,height=16,columns=('#0','#1','#2','#3','#4','#5','#6','#7','#8','#9'), show='headings') # con '#6' se crean las columnas, # show='headings' <- oculta el ID de la tabla
    #tree.place(x=0,y=130)
    tree.column('#0', width=40)
    tree.heading('#0', text='ID', anchor=CENTER)
    tree.column('#1', width=120)
    tree.heading('#1', text='Code', anchor=CENTER)
    tree.column('#2', width=70)
    tree.heading('#2', text='L Max', anchor=CENTER)
    tree.column('#3', width=70)
    tree.heading('#3', text='L Min', anchor=CENTER)
    tree.column('#4', width=70)
    tree.heading('#4', text='A Max', anchor=CENTER)
    tree.column('#5', width=70)
    tree.heading('#5', text='A Min', anchor=CENTER)
    tree.column('#6', width=70)
    tree.heading('#6', text='B Max', anchor=CENTER)
    tree.column('#7', width=70)
    tree.heading('#7', text='B Min', anchor=CENTER)
    tree.column('#8', width=70)
    tree.heading('#8', text='L Ref', anchor=CENTER)
    tree.column('#9', width=70)
    tree.heading('#9', text='A Ref', anchor=CENTER)
    tree.column('#10', width=70)
    tree.heading('#10', text='B Ref', anchor=CENTER)
    tree.place(x=20,y=240)

    ## Seleccionar usando click para modificar registros 
    def seleccionarUsandoClick(event): # Event porque hay que pasar un evento
        item=tree.identify('item',event.x,event.y)
        miId.set(tree.item(item,'text'))
        Cod_Bulk.set(tree.item(item,'values')[0])
        L_max.set(tree.item(item,'values')[1])
        L_min.set(tree.item(item,'values')[2])
        A_max.set(tree.item(item,'values')[3])
        A_min.set(tree.item(item,'values')[4])
        B_max.set(tree.item(item,'values')[5])
        B_min.set(tree.item(item,'values')[6])
        L_patron.set(tree.item(item,'values')[7])
        A_patron.set(tree.item(item,'values')[8])
        B_patron.set(tree.item(item,'values')[9])
    # hay que ligar esta función a la TABLA
    tree.bind('<Double-1>',seleccionarUsandoClick)

## Creando etiquetas y cajas de texto
# Entradas de los datos
    e1=Entry(insertdata, textvariable=miId)
    #________________________________
    l2=Label(insertdata, text='Code',width=11,  relief='sunken',borderwidth=4, font='Verdana 12 bold')
    l2.place(x=260,y=40)
    e2=Entry(insertdata, textvariable=Cod_Bulk, width=19, bd= 5, justify=CENTER)
    e2.place(x=262,y=70)
    #________________________________
    #________________________________
    l3=Label(insertdata, text='ΔL Maximun',width=10, relief="raised",borderwidth=4, font='Verdana 10 bold')
    l3.place(x=20,y=120)
    e3=Entry(insertdata, textvariable=L_max, width=15,justify=CENTER)
    e3.place(x=22,y=145)

    l4=Label(insertdata, text='ΔL Minimun',width=10, relief="raised",borderwidth=4, font='Verdana 10 bold')
    l4.place(x=20 ,y=170)
    e4=Entry(insertdata, textvariable=L_min, width=15,justify=CENTER)
    e4.place(x=22,y=195)
    #________________________________
    #________________________________
    l5=Label(insertdata, text='ΔA Maximun',width=10, relief="raised",borderwidth=4, font='Verdana 10 bold')
    l5.place(x=150 ,y=120)
    e5=Entry(insertdata, textvariable=A_max, width=15,justify=CENTER)
    e5.place(x=152,y=145)

    l5=Label(insertdata, text='ΔA Minimun',width=10, relief="raised",borderwidth=4, font='Verdana 10 bold')
    l5.place(x=150 ,y=170)
    e5=Entry(insertdata, textvariable=A_min, width=15,justify=CENTER)
    e5.place(x=152,y=195)
    #________________________________
    #________________________________
    l6=Label(insertdata, text='ΔB Maximun',width=10, relief="raised",borderwidth=4, font='Verdana 10 bold')
    l6.place(x=280 ,y=120)
    e6=Entry(insertdata, textvariable=B_max, width=15,justify=CENTER)
    e6.place(x=282,y=145)

    l6=Label(insertdata, text='ΔB Minimun',width=10, relief="raised",borderwidth=4, font='Verdana 10 bold')
    l6.place(x=280 ,y=170)
    e6=Entry(insertdata, textvariable=B_min, width=15,justify=CENTER)
    e6.place(x=282,y=195)
    #________________________________
    #________________________________L-A-B PATRON
    l7=Label(insertdata, text='L-A-B Reference',width=34, height=2,relief="raised",borderwidth=5,font='Verdana 10 bold', bg="honeydew3")
    l7.place(x=420 ,y=120)

    l8=Label(insertdata, text='L',width=6, height=1,font='Verdana 12 bold',relief="raised",borderwidth=3)
    l8.place(x=460 ,y=165)
    e8=Entry(insertdata, textvariable=L_patron, width=11,justify=CENTER)
    e8.place(x=460, y=195)

    l9=Label(insertdata, text='A',width=6, height=1,font='Verdana 12 bold',relief="raised",borderwidth=3)
    l9.place(x=540 ,y=165)
    e9=Entry(insertdata, textvariable=A_patron, width=11,justify=CENTER)
    e9.place(x=540, y=195)

    l10=Label(insertdata, text='B',width=6, height=1,font='Verdana 12 bold',relief="raised",borderwidth=3)
    l10.place(x=620 ,y=165)
    e10=Entry(insertdata, textvariable=B_patron, width=11,justify=CENTER)
    e10.place(x=620, y=195)
    #________________________________

    #################################### Creando botones
    # Boton Crear registro
    b1= Button(insertdata, text='Save\nData', command=crear,width=7, cursor = 'hand2',height=5,font='Verdana 8 bold',relief=GROOVE ,borderwidth=6, bg='pale green')
    b1.place(x=460,y=30)
    # Boton modificar registro
    b2= Button(insertdata, text='Modify\nData', command=actualizar,width=7, cursor = 'hand2',height=5,font='Verdana 8 bold',relief=GROOVE ,borderwidth=6, bg='light cyan')
    b2.place(x=540,y=30)
    # Boton mostrar lista
    '''b3= Button(insertdata, text='Mostrar\nlista', command=mostrar,width=7, height=5,font='Verdana 8 bold',relief=GROOVE ,borderwidth=6, bg='LightSkyBlue1')
    b3.place(x=580,y=30)'''
    # Boton eliminar registro
    b4= Button(insertdata, text='Clear Data', command=borrar, bg='mistyrose')
    b4.place(x=25,y=600)





###################################################################
###################################################################
# Crear base de datos con los resultados de la consulta

    def crear_conexion_BDCC():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        #Crear tabla
        try:
            miCursor.execute("""
                    CREATE TABLE IF NOT EXISTS consulta(
                    ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                    bulk_bar INTEGER NOT NULL,
                    op_var INTEGER NOT NULL,
                    l_var FLOAT NOT NULL,
                    a_var FLOAT NOT NULL,
                    b_bar FLOAT NOT NULL,
                    fecha TEXT NOT NULL
                    );
                """)
            # Cuadro de texto o dialogo para mostrar dicha información
            messagebox.showinfo('CONEXION', 'Base de datos creada exitosamente')
        except:
            messagebox.showinfo('CONEXION', 'Conexión exitosa con la Base de datos')



    # Función mostrar los datos al crear un registro y que se vea en la lista de registros
    def mostrarconsulta():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        registros=tree5.get_children() # Permite extraer todos los datos de la tabla
        for elemento in registros:
            tree5.delete(elemento) # Este borrado es solo en la parte de la ventana donde colocamos las variables
        # Try y catch para llenar la tabla con los valores extraidos de la BD
        try:
            miCursor.execute('SELECT * FROM consulta')
            for row in miCursor:
                tree5.insert('',0,text=row[0],value=(row[1],row[2],row[3],row[4],row[5],row[6]))
        except:
            pass


    # Crear la tabla  tree5
    tree5=ttk.Treeview(insertdata,height=16,columns=('#0','#1','#2','#3','#4','#5'), show='headings') # con '#6' se crean las columnas, # show='headings' <- oculta el ID de la tabla
    #tree.place(x=0,y=130)
    tree5.column('#0', width=40)
    tree5.heading('#0', text='ID', anchor=CENTER)
    tree5.column('#1', width=120)
    tree5.heading('#1', text='Code', anchor=CENTER)
    tree5.column('#2', width=70)
    tree5.heading('#2', text='Lot', anchor=CENTER)
    tree5.column('#3', width=70)
    tree5.heading('#3', text='L ref', anchor=CENTER)
    tree5.column('#4', width=70)
    tree5.heading('#4', text='A ref', anchor=CENTER)
    tree5.column('#5', width=70)
    tree5.heading('#5', text='B ref', anchor=CENTER)
    tree5.column('#6', width=70)
    tree5.heading('#6', text='Date', anchor=CENTER)
    tree5.place(x=800,y=240)

    ## Boton mostrar listas
    b3= Button(insertdata, text='Show\nlist', command=lambda:[mostrar(),mostrarconsulta()],width=7, height=5,font='Verdana 8 bold',relief=GROOVE ,borderwidth=6, bg='LightSkyBlue1')
    b3.place(x=620,y=30)



    ########################################################
    ########################################################
    ########################################################
    ########################################################
    #### CONSULTA DE LOS RESULTADOS DE CALIDAD POR LA OP
    
    op_var = StringVar()
    
    titulo_OP = Label(insertdata,text='Search results',justify='center',relief=RIDGE,borderwidth=8,width=50,bg='azure',height=2,font='Verdana 8 bold')
    titulo_OP.place(x=810 ,y=15)
    consulta_OP = Label(insertdata, text='Insert Lot',justify='center',relief=RIDGE,borderwidth=4)
    consulta_OP.place(x=810 ,y=70)
    consulta_OP_dat= Entry(insertdata, textvariable=op_var, width=23, justify=CENTER,relief=GROOVE,borderwidth=2)
    consulta_OP_dat.place(x=890,y=70)

    def mostrar_resultado_op():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        registros=tree6.get_children()
        var_op =int(consulta_OP_dat.get())
        try:
            miCursor.execute("SELECT * FROM consulta WHERE op_var=%s" % var_op)
            for row in miCursor:
                tree6.insert('',0,text=row[0],value=(row[1],row[2],row[3],row[4],row[5],row[6]))
        except:
            messagebox.showwarning("ADVERTENCIA","Ocurrió un error al realizar la consulta")
            pass


    boton_op= Button(insertdata, command= mostrar_resultado_op, text='click to\n Search',width=17, height=2,font='Verdana 8 bold',relief=GROOVE ,borderwidth=6, bg='thistle', cursor = 'hand2')
    boton_op.place(x=1050,y=60)

    tree6=ttk.Treeview(insertdata, height=1,columns=('#0','#1','#2','#3','#4','#5'), show='headings')
    tree6.column('#0', width=40)
    tree6.heading('#0', text='ID', anchor=CENTER)
    tree6.column('#1', width=120)
    tree6.heading('#1', text='Code', anchor=CENTER)
    tree6.column('#2', width=70)
    tree6.heading('#2', text='Lot', anchor=CENTER)
    tree6.column('#3', width=70)
    tree6.heading('#3', text='L ref', anchor=CENTER)
    tree6.column('#4', width=70)
    tree6.heading('#4', text='A ref', anchor=CENTER)
    tree6.column('#5', width=70)
    tree6.heading('#5', text='B ref', anchor=CENTER)
    tree6.column('#6', width=70)
    tree6.heading('#6', text='Date', anchor=CENTER)
    tree6.place(x=800,y=120)



    ########################################################
    ########################################################    ########################################################
    ########################################################
    ### Barras de MENU


    def regresar():
        insertdata.withdraw()
        logging.deiconify()

    botton_regresar = Button(insertdata, text = 'Back',command = regresar, cursor = 'hand2', relief=RAISED ,bg= 'grey50',font =('Candara',20, 'bold'))
    botton_regresar.place(x=140,y=600)

    menubar=Menu(insertdata)
    menubasedat=Menu(menubar,tearoff=0)
    menubasedat.add_command(label="Create/conect BD", command=conexionBD)
    menubasedat.add_command(label="Create/conect BDCC", command=crear_conexion_BDCC)
    

    menubasedat.add_command(label="Delete BD", command=eliminarBBDD)
    menubasedat.add_command(label="Exit", command=salirAplicacion)
    # Vicular las 3 opciones del menu a la barra de menus
    menubar.add_cascade(label='Start', menu=menubasedat)

    ayudamenu=Menu(menubar,tearoff=0)
    ayudamenu.add_command(label="Clean Field", command=limpiarCampos)
    ayudamenu.add_command(label="About App", command=mensaje)
    menubar.add_cascade(label='Help', menu=ayudamenu) 

    insertdata.config(menu=menubar)
    insertdata.mainloop()
    

########### Ingresar a la pagina donde calidad ingresa los datos
########### Ingresar a la pagina donde calidad ingresa los datos
########### Ingresar a la pagina donde calidad ingresa los datos
########### Ingresar a la pagina donde calidad ingresa los datos

def c_calidad():
    global consultar
    logging.withdraw()
    consultar = tk.Toplevel()
    consultar.title('Module B')
    consultar.geometry('1300x800')
    consultar.config(bg='grey90')
    consultar.iconbitmap('Captura.ico')

    
    img3 = Image.open('tkinter.png')
    img3 = img3.resize((180,90), Image.ANTIALIAS)
    img3 = ImageTk.PhotoImage(img3)
    img_label = Label(consultar, image = img3, relief=SUNKEN,borderwidth=4)
    img_label.place(x=5,y=40)

    ## Variables para ingresar a la base de datos de resultados de calidad

    Id = StringVar()
    Bulk_var = StringVar()
    Op_var = StringVar()
    L_var = StringVar()
    A_var = StringVar()
    B_var = StringVar()
    Fecha_var = StringVar()
    #________________________________
    encabezado = Label(consultar, text='Enter Results',width=60, height=2,font='Verdana 13 bold',relief=GROOVE ,borderwidth=6, bg='light cyan',justify=CENTER)
    encabezado.place(x=300,y=10)
    #________________________________
    cod_bul_mtra = Label(consultar, text='Enter a Code', width=19,justify='center',relief=RIDGE,borderwidth=4)
    cod_bul_mtra.place(x=200,y=70)
    cod_bul_mtra_dat= Entry(consultar, textvariable=Bulk_var, width=23, justify=CENTER,relief=GROOVE,borderwidth=2)
    cod_bul_mtra_dat.place(x=200,y=100)
    #________________________________
    patron_L=Label(consultar, text='Enter L Ref' , width=15,justify='center', relief=RIDGE, borderwidth=4)
    patron_L.place(x=360,y=70)
    patron_L_dat=Entry(consultar,textvariable= L_var, width=17, justify=CENTER,relief=GROOVE,borderwidth=2)
    patron_L_dat.place(x=361,y=100)
    #________________________________
    patron_A=Label(consultar, text='Enter A Ref', width=17,justify='center',relief=RIDGE,borderwidth=4)
    patron_A.place(x=490,y=70)
    patron_A_dat=Entry(consultar, textvariable=A_var, width=20, justify=CENTER,relief=GROOVE,borderwidth=2)
    patron_A_dat.place(x=491,y=100)
    #________________________________
    patron_B=Label(consultar, text='Enter B Ref',width=17,justify='center',relief=RIDGE,borderwidth=4)
    patron_B.place(x=630,y=70)
    patron_B_dat=Entry(consultar, textvariable=B_var, width=20, justify=CENTER,relief=GROOVE,borderwidth=2)
    patron_B_dat.place(x=631,y=100)
    #________________________________
    op =Label(consultar, text='Lot',width=17,justify='center',relief=RIDGE,borderwidth=4)
    op.place(x=770,y=70)
    op_dat=Entry(consultar, textvariable=Op_var,width=20, justify=CENTER,relief=GROOVE,borderwidth=2)
    op_dat.place(x=771,y=100)
    #________________________________
    # CAMPO PARA FECHA
    fecha =Label(consultar, text='Analysis date',width=12,justify='center',relief=RIDGE,borderwidth=4)
    fecha.place(x=905,y=70)
    fecha_dat=Entry(consultar, textvariable=Fecha_var,width=14, justify=CENTER,relief=GROOVE,borderwidth=2)
    fecha_dat.place(x=906,y=100)


    #### INSERTAR TABLA DE LOS DATOS CONSULTADOS CON EL CODIGO BULK#####

    # Función mostrar los datos al crear un registro y que se vea en la lista de registros
    def mostrartree2():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        registros=tree2.get_children() # Permite extraer todos los datos de la tabla

        var = int(cod_bul_mtra_dat.get())
        #print('el codigo bulk es', var)
        # Este borrado es solo en la parte de la ventana donde colocamos las variables
        # Try y catch para llenar la tabla con los valores extraidos de la BD
        try:
            miCursor.execute("SELECT * FROM especificacion WHERE CodBulk=%s" % var)
            for row in miCursor:
                tree2.insert('',0,text=row[0],value=(row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10]))
        except:
            pass


    # Boton mostrar CONINCIDENCIA DE CODÍGO BULK mostrartree2
    b3= Button(consultar, text='Show results', command=mostrartree2, cursor = 'hand2',width=16, height=2,font='Verdana 10 bold',relief=RAISED ,borderwidth=6, bg='LightSkyBlue1',justify=CENTER)
    b3.place(x=1010,y=70)

    # Crear la tabla 
    tree2=ttk.Treeview(consultar,height=1, columns=('#0','#1','#2','#3','#4','#5','#6','#7','#8','#9'), show='headings') # con '#6' se crean las columnas
    #tree.place(x=0,y=130)
    tree2.column('#0', width=50)
    tree2.heading('#0', text='ID', anchor=CENTER)
    tree2.column('#1', width=130)
    tree2.heading('#1', text='Code', anchor=CENTER)
    tree2.column('#2', width=80)
    tree2.heading('#2', text='L Max', anchor=CENTER)
    tree2.column('#3', width=80)
    tree2.heading('#3', text='L Min', anchor=CENTER)
    tree2.column('#4', width=80)
    tree2.heading('#4', text='A Max', anchor=CENTER)
    tree2.column('#5', width=80)
    tree2.heading('#5', text='A Min', anchor=CENTER)
    tree2.column('#6', width=80)
    tree2.heading('#6', text='B Max', anchor=CENTER)
    tree2.column('#7', width=80)
    tree2.heading('#7', text='B Min', anchor=CENTER)
    tree2.column('#8', width=80)
    tree2.heading('#8', text='L Ref', anchor=CENTER)
    tree2.column('#9', width=80)
    tree2.heading('#9', text='A Ref', anchor=CENTER)
    tree2.column('#10', width=80)
    tree2.heading('#10', text='B Ref', anchor=CENTER)
    tree2.place(x=250,y=140)


    ## OPERACIONES LOGICAS PARA CALCULAR DELTA _L_ DE PATRON MTRA especificaciones

    canvas_1 = Canvas(consultar, width=900,height=300, bg ='white', relief=GROOVE, borderwidth=10)
    canvas_1.pack()

    def comparardeltaL():
        engine = sqlalchemy.create_engine('sqlite:///base_Frag.db')
        df=pd.read_sql('especificacion', engine)
        v1 = float(cod_bul_mtra_dat.get())
        L = float(patron_L_dat.get())
        f = df[df['CodBulk'] == v1]
        resl = f['L_Patron'].values - L

        restaL = ('Diferencia ΔL', resl)
        label_restaL = Label(consultar, text=restaL, bg = 'orange', fg='white', font='Verdana 8 bold',relief=RIDGE ,borderwidth=4,justify=CENTER)
        canvas_1.create_window(160,110, window=label_restaL)

    button_1 = Button(consultar, text='Click to show ΔL',command=comparardeltaL, bg = '#856ff8', fg='white', font='Verdana 10 bold',relief=RAISED ,borderwidth=6,justify=CENTER)
    canvas_1.create_window(160,70, window=button_1)

    # Fución Logica para comparar delta B
    def siLpasa():
        engine = sqlalchemy.create_engine('sqlite:///base_Frag.db')
        df=pd.read_sql('especificacion', engine)
        v1 = float(cod_bul_mtra_dat.get())
        L = float(patron_L_dat.get())
        f = df[df['CodBulk'] == v1]
        resl_pas = f['L_Patron'].values - L

        if (resl_pas-0.00001)<=f['L_Max'].values and (resl_pas+0.00001)>= f['L_Min'].values:
            #print('PASS')
            label_pas_L = ('Pass')
            label_resta_L = Label(consultar, text=label_pas_L, bg ='green4', fg='white',justify='center',relief=RIDGE,borderwidth=4, width=10, height=3,font='Century 16 bold')
            canvas_1.create_window(160,220, window=label_resta_L)
        else:
            #print('FAIL')
            label_pas_L = ('Fail ')
            label_resta_L = Label(consultar, text=label_pas_L, bg = 'red2',  fg='white',justify='center',relief=RIDGE,borderwidth=4, width=10, height=3, font='Century  16 bold')
            canvas_1.create_window(160,220, window=label_resta_L)

    button_4 = Button(consultar, text='Concept',command=siLpasa, bg = 'medium purple', fg='white',font='Verdana 8 bold',relief=RAISED ,borderwidth=4,justify=CENTER)
    canvas_1.create_window(160,150, window=button_4)

    ##_______________________________________________________________
    ## OPERACIONES LOGICAS PARA CALCULAR DELTA _A_ DE PATRON MTRA especificaciones

    def comparardeltaA():
        engine = sqlalchemy.create_engine('sqlite:///base_Frag.db')
        df=pd.read_sql('especificacion', engine)
        v1 = float(cod_bul_mtra_dat.get())
        L = float(patron_A_dat.get())
        f = df[df['CodBulk'] == v1]
        resA = f['A_Patron'].values - L

        restaA = ('Diferencia ΔA', resA)
        label_restaA = Label(consultar, text=restaA, bg = 'orange', fg='white',font='Verdana 8 bold',relief=RIDGE ,borderwidth=4,justify=CENTER)
        canvas_1.create_window(450,110, window=label_restaA)

    button_2 = Button(consultar, text='Click to show ΔA',command=comparardeltaA, bg = '#856ff8', fg='white', font='Verdana 10 bold',relief=RAISED ,borderwidth=6,justify=CENTER)
    canvas_1.create_window(450,70, window=button_2)

    # Fución Logica para comparar delta A
    def siApasa():
        engine = sqlalchemy.create_engine('sqlite:///base_Frag.db')
        df=pd.read_sql('especificacion', engine)
        v1 = float(cod_bul_mtra_dat.get())
        L = float(patron_A_dat.get())
        f = df[df['CodBulk'] == v1]
        resA_pas = f['A_Patron'].values - L
        if (resA_pas-0.00001)<=f['A_Max'].values and (resA_pas+0.00001)>= f['A_Min'].values:
            #print('PASS')
            label_pas_A = ('Pass')
            label_resta_A = Label(consultar, text=label_pas_A, bg = 'green4', fg='white',justify='center',relief=RIDGE,borderwidth=4, width=10, height=3,font='Century 16 bold')
            canvas_1.create_window(450,220, window=label_resta_A)
        else:
            #print('FAIL')
            label_pas_A = ('Fail')
            label_resta_A = Label(consultar, text=label_pas_A, bg = 'red2', fg='white',justify='center',relief=RIDGE,borderwidth=4, width=10, height=3,font='Century 16 bold')
            canvas_1.create_window(450,220, window=label_resta_A)

    button_5 = Button(consultar, text='Concept',command=siApasa, bg = 'medium purple', fg='white',font='Verdana 8 bold',relief=RAISED ,borderwidth=4,justify=CENTER)
    canvas_1.create_window(450,150, window=button_5)

    ##_______________________________________________________________
    ##_______________________________________________________________
    ## OPERACIONES LOGICAS PARA CALCULAR DELTA _B_ DE PATRON MTRA especificaciones

    def comparardeltaB():
        engine = sqlalchemy.create_engine('sqlite:///base_Frag.db')
        df=pd.read_sql('especificacion', engine)
        v1 = float(cod_bul_mtra_dat.get())
        L = float(patron_B_dat.get())
        f = df[df['CodBulk'] == v1]
        resB = f['B_Patron'].values - L

        restaB = ('Diferencia ΔB', resB)
        label_restaB= Label(consultar, text=restaB , bg = 'orange', fg='white', font='Verdana 8 bold',relief=RIDGE ,borderwidth=4,justify=CENTER)
        canvas_1.create_window(750,110, window=label_restaB)

    button_3 = Button(consultar, text='Click to show ΔB',command=comparardeltaB, bg = '#856ff8', fg='white', font='Verdana 10 bold',relief=RAISED ,borderwidth=6,justify=CENTER)
    canvas_1.create_window(750,70, window=button_3)

    # Fución Logica para comparar delta B
    def siBpasa():
        engine = sqlalchemy.create_engine('sqlite:///base_Frag.db')
        df=pd.read_sql('especificacion', engine)
        v1 = float(cod_bul_mtra_dat.get())
        L = float(patron_B_dat.get())
        f = df[df['CodBulk'] == v1] 
        resB_pas = f['B_Patron'].values - L

        #if f['B_Min'].values <= (resB_pas) <= f['B_Max'].values:
        if (resB_pas-0.00001)<=f['B_Max'].values and (resB_pas+0.00001)>= f['B_Min'].values:
        #if (resB_pas==f['B_Max'].values and resB_pas== f['B_Min'].values) or (resB_pas<f['B_Max'].values and resB_pas> f['B_Min'].values):
            #print('PASS')
            label_pas_B = ('Pass')
            label_resta_B = Label(consultar, text=label_pas_B, bg = 'green4', fg='white',justify='center',relief=RIDGE,borderwidth=4, width=10, height=3,font='Century 16 bold')
            canvas_1.create_window(750,220, window=label_resta_B)
        else:
            #print('FAIL')
            label_pas_B = ('Fail')
            label_resta_B = Label(consultar, text=label_pas_B, bg = 'red2', fg='white',justify='center',relief=RIDGE,borderwidth=4, width=10, height=3,font='Century 16 bold')
            canvas_1.create_window(750,220, window=label_resta_B)

    button_6 = Button(consultar, text='Concept',command=siBpasa, bg = 'medium purple', fg='white',font='Verdana 8 bold',relief=RAISED ,borderwidth=4,justify=CENTER)
    canvas_1.create_window(750,150, window=button_6)


    canvas_1.place(x=220, y=250)


    # Metodo para crear 
    def crearconsulta():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        try:
            datos=Bulk_var.get(), Op_var.get(), L_var.get(), A_var.get(), B_var.get(), Fecha_var.get()
            miCursor.execute('INSERT INTO consulta VALUES(NULL,?,?,?,?,?,?)',(datos) )
            miConexion.commit() # Commit para que se actualicen los cambios en la tabla
        except:
            messagebox.showwarning('ADVERTENCIA","Ocurrió un error al crear el registro, verifique conexión con BBDD')
            pass
        #limpiarCampos()

    ## CREAR BOTON PARA CALCULAR TOD0

    boton_all = Button(consultar, text='Compute', 
                        command=lambda:[crearconsulta(),comparardeltaL(),
                                        siLpasa(),comparardeltaA(),siApasa(),
                                        comparardeltaB(),siBpasa()], cursor = 'hand1',
                                        width=30, height=1,font='Verdana 13 bold',relief=GROOVE ,
                                        borderwidth=6, bg="coral1",fg="white",justify=CENTER)

    boton_all.place(x=500,y=195)


    def regresar():
        consultar.withdraw()
        logging.deiconify()

    botton_regresar = Button(consultar, text = 'Back',command = regresar, cursor = 'hand2', relief=RAISED ,bg= 'grey50',font =('Candara',20, 'bold'))
    botton_regresar.place(x=40,y=600)

    ########################################################
    ########################################################
    ########################################################
    ########################################################
    #### CONSULTA DE LOS RESULTADOS DE CALIDAD POR LA OP
    
    op_var = StringVar()
    
    titulo_OP = Label(consultar,text='Search results',justify='center',relief=RIDGE,borderwidth=8,width=47,bg='azure',height=2,font='Verdana 8 bold')
    titulo_OP.place(x=220 ,y=590)
    consulta_OP = Label(consultar, text='Insert Lot',justify='center',relief=RIDGE,borderwidth=4)
    consulta_OP.place(x=220 ,y=640)
    consulta_OP_dat= Entry(consultar, textvariable=op_var, width=23, justify=CENTER,relief=GROOVE,borderwidth=2)
    consulta_OP_dat.place(x=300,y=640)

    def mostrar_resultado_op():
        miConexion=sqlite3.connect('base_Frag.db')
        miCursor=miConexion.cursor()
        registros=tree6.get_children()
        var_op =int(consulta_OP_dat.get())
        try:
            miCursor.execute("SELECT * FROM consulta WHERE op_var=%s" % var_op)
            for row in miCursor:
                tree6.insert('',0,text=row[0],value=(row[1],row[2],row[3],row[4],row[5],row[6]))
        except:
            messagebox.showwarning("ADVERTENCIA","Ocurrió un error al realizar la consulta")
            pass

    boton_op= Button(consultar, command= mostrar_resultado_op, text='click to \n Search',width=17, height=2,font='Verdana 8 bold',relief=GROOVE ,borderwidth=6, bg='thistle', cursor = 'hand2')
    boton_op.place(x=455,y=635)

    tree6=ttk.Treeview(consultar, height=1,columns=('#0','#1','#2','#3','#4','#5'), show='headings')
    tree6.column('#0', width=40)
    tree6.heading('#0', text='ID', anchor=CENTER)
    tree6.column('#1', width=120)
    tree6.heading('#1', text='Insert Lot', anchor=CENTER)
    tree6.column('#2', width=70)
    tree6.heading('#2', text='Lot', anchor=CENTER)
    tree6.column('#3', width=70)
    tree6.heading('#3', text='L Ref', anchor=CENTER)
    tree6.column('#4', width=70)
    tree6.heading('#4', text='A Ref', anchor=CENTER)
    tree6.column('#5', width=70)
    tree6.heading('#5', text='B Ref', anchor=CENTER)
    tree6.column('#6', width=70)
    tree6.heading('#6', text='Date', anchor=CENTER)
    tree6.place(x=660,y=600)

    ########################################################
    ########################################################    ########################################################
    ########################################################


    consultar.mainloop()

#### CREAR HOJA DE DATAFRAME DE I&D
#### CREAR HOJA DE DATAFRAME DE I&D

def df_id():
    global dfid 
    logging.withdraw()
    dfid = tk.Toplevel()
    dfid.title('Datos Patrones ID')
    dfid.geometry("1200x300")
    dfid.config(bg='grey90')
    dfid.iconbitmap('Captura.ico')

    # Actualizar datos ID
    canvas_5 = Canvas(dfid , width=900,height=200, bg ='white', relief=GROOVE, borderwidth=10)
    canvas_5.pack()    

    def actualizarDF():
        engine = sqlalchemy.create_engine('sqlite:///base_Frag.db')
        df=pd.read_sql('especificacion', engine)
        pt = Table(canvas_5 , dataframe=df,width=800 ,height=200)
        # pt = Table(Dataframe , dataframe=df,width=1200 ,height=1200, showtoolbar=True, showstatusbar=True)
        pt.show()

    def regresar():
        dfid.withdraw()
        logging.deiconify()


    botton_regresar = Button(dfid, text = 'back',command = regresar, cursor = 'hand2', relief=RAISED ,bg= 'grey50',font =('Candara',15, 'bold'))
    botton_regresar.place(x=30,y=120)

        # Se inserta el boton de dataFrame para actualizarlo en la pestaña Crear datos
    ee= Button(dfid, text='Actualizar \n Datos I&D', cursor = 'hand2',command=actualizarDF,width=9, height=5,font='Verdana 8 bold',relief=GROOVE ,borderwidth=6, bg='DarkGoldenrod2')
    ee.place(x=30,y=30)

    canvas_5.create_window(500,500)
    

    dfid.mainloop()


#### CREAR HOJA DE DATAFRAME DE CALIDAD
#### CREAR HOJA DE DATAFRAME DE CALIDAD

def df_cc():
    global dfcc
    logging.withdraw()
    dfcc = tk.Toplevel()
    dfcc.title('Datos Control Calidad')
    dfcc.geometry("1200x300")
    dfcc.config(bg='grey90')
    dfcc.iconbitmap('Captura.ico')


    # Actualizar datos ID
    canvas_6 = Canvas(dfcc , width=900,height=200, bg ='white', relief=GROOVE, borderwidth=10)
    canvas_6.pack()    

    def actualizarDFcc():

        conexion = sqlite3.connect('base_Frag.db')
        dataCC = pd.read_sql('SELECT * FROM consulta', conexion)
        dataCC1 = Table(canvas_6 , dataframe=dataCC,width=800 ,height=200)
        dataCC1.show()


    def regresar():
        dfcc.withdraw()
        logging.deiconify()

    botton_regresar = Button(dfcc, text = 'back',command = regresar, cursor = 'hand2', relief=RAISED ,bg= 'grey50',font =('Candara',15, 'bold'))
    botton_regresar.place(x=30,y=120)

        # Se inserta el boton de dataFrame para actualizarlo en la pestaña Crear datos
    ee22= Button(dfcc, text='Actualizar \n Datos CC', command=actualizarDFcc,width=9,cursor = 'hand2', height=5,font='Verdana 8 bold',relief=GROOVE ,borderwidth=6, bg='DarkGoldenrod2')
    ee22.place(x=30,y=30)

    canvas_6.create_window(500,500)
    
    dfid.mainloop()



    ########################################################
    ########################################################
    ########################################################
    ########################################################
    #### INGRESAR MODULO DE MATERIA PRIMA 
        ########################################################
    ########################################################
    ########################################################tk.Toplevel
def materiaprima():
    global mp
    logging.withdraw()
    mp = tk.Toplevel()
    mp.title('Análisis Colorantes')
    mp.geometry('780x500+280+100')
    mp.config(bg='gray90')
    mp.iconbitmap('Captura.ico')
    #mp.resizable(width=0, height=0)
    mp.resizable(1, 1)
    canvas1 = Canvas(mp, width = 350, height = 200) 
    canvas1.pack()  

        #################################################
        #################################################
        #### IMAGEN 1 #####
    Limg1 = StringVar()
    Aimg1 = StringVar()
    Bimg1 = StringVar()

    ##_____________________________________________##
    # Ingresar valores LAB mediante los botones para IMAGEN 1
    encabezado = Label(mp, text='Ingrese coordenadas CIE L*a*b* colorante',width=60, height=2,font='Verdana 13 bold',relief=GROOVE ,borderwidth=6,justify=CENTER)
    encabezado.place(x=22,y=10)


    limagen1 = Label(mp, text = "Valor L referencia",justify='center',relief=RIDGE,borderwidth=4, width=14)
    limagen1.place(x = 10, y=70)
    limagen1_dat = Entry(mp, textvariable = Limg1, width=17, justify=CENTER,relief=GROOVE,borderwidth=2)
    limagen1_dat.place(x=10, y = 100) 

    aimagen1 = Label(mp, text = "Valor A referencia",justify='center',relief=RIDGE,borderwidth=4, width=14)
    aimagen1.place(x = 130, y=70)
    aimagen1_dat = Entry(mp, textvariable = Aimg1, width=17, justify=CENTER,relief=GROOVE,borderwidth=2)
    aimagen1_dat.place(x=130, y = 100)

    bimagen1 = Label(mp, text = "Valor B referencia",justify='center',relief=RIDGE,borderwidth=4, width=14)
    bimagen1.place(x = 250, y=70)
    bimagen1_dat = Entry(mp, textvariable = Bimg1, width=17, justify=CENTER,relief=GROOVE,borderwidth=2)
    bimagen1_dat.place(x=250, y = 100)
        
            ## Sección para ingresar el color
    global imagenoscura
    
    def colorimg1():
        global imgtk
        L = float(Limg1.get())
        A = float(Aimg1.get())
        B = float(Bimg1.get())
            
            # Ingresar los valores LAB y convertirlos a RGB
        colorRGB = lab2rgb([L,A,B])*255
            # Convertir RGB a BGR cambiando la primera conlumna del array por la última con el comando de numpy np.flip
            #colorRGB = np.flip(colorRGB) # CONVERTIR DE RGB A BGR
        imagenoscura = np.zeros((170,290,3), np.uint8)
        alto, largo, canales = imagenoscura.shape
        for i in range(alto):
            for j in range(largo):
                pixel = imagenoscura[i,j]
                if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
                    imagenoscura[i,j] = colorRGB # B,G,R

        im = Image.fromarray(imagenoscura)
        imgtk = ImageTk.PhotoImage(image=im)
            
            #img_label = Label(win, image= imgtk,relief=SUNKEN,borderwidth=4)
            #img_label.place(x = 15,y = 120).pack()
        Label(mp, image= imgtk,relief=SUNKEN,borderwidth=4,width=320, height=170).place(x = 25,y = 195).grid(row=0, column=0)
        canvas1.create_image(image=imgtk)

    canvas1.place(x=10, y=180)

    botonimg1 = Button(mp, text = "Show color 1", width=25, command = colorimg1,fg='black',font='Verdana 8 bold',relief=RAISED ,borderwidth=4,justify=CENTER,cursor = 'hand2')
    botonimg1.place(x=80, y =130)


#### IMAGEN 2 #####

    canvas2 = Canvas(mp, width = 350, height = 200) 
    canvas2.pack()  

    Limg2 = StringVar()
    Aimg2 = StringVar()
    Bimg2 = StringVar()

    ##_____________________________________________##
    # Ingresar valores LAB mediante los botones para IMAGEN 2
    limagen2 = Label(mp, text = "Valor L muestra",justify='center',relief=RIDGE,borderwidth=4,width=14)
    limagen2.place(x = 420, y=70)
    limagen2_dat = Entry(mp, textvariable = Limg2, width=17, justify=CENTER,relief=GROOVE,borderwidth=2)
    limagen2_dat.place(x=420, y = 100) 

    aimagen2 = Label(mp, text = "Valor A muestra",justify='center',relief=RIDGE,borderwidth=4, width=14)
    aimagen2.place(x = 540, y=70)
    aimagen2_dat = Entry(mp, textvariable = Aimg2, width=17, justify=CENTER,relief=GROOVE,borderwidth=2)
    aimagen2_dat.place(x=540, y = 100)

    bimagen2 = Label(mp, text = "Valor B muestra",justify='center',relief=RIDGE,borderwidth=4, width=14)
    bimagen2.place(x = 660, y=70)
    bimagen2_dat = Entry(mp, textvariable = Bimg2, width=17, justify=CENTER,relief=GROOVE,borderwidth=2)
    bimagen2_dat.place(x=660, y = 100)


    global imagenoscura2
    
    def colorimg2():
        global imgtk2
        L2 = float(Limg2.get())
        A2 = float(Aimg2.get())
        B2 = float(Bimg2.get())

        # Ingresar los valores LAB y convertirlos a RGB
        colorRGB2 = lab2rgb([L2,A2,B2])*255
        # Convertir RGB a BGR cambiando la primera conlumna del array por la última con el comando de numpy np.flip
        #colorRGB = np.flip(colorRGB) # CONVERTIR DE RGB A BGR
        imagenoscura2 = np.zeros((170,290,3), np.uint8)
        alto2, largo2, canales = imagenoscura2.shape
        for i in range(alto2):
            for j in range(largo2):
                pixel2 = imagenoscura2[i,j]
                if pixel2[0] == 0 and pixel2[1] == 0 and pixel2[2] == 0:
                    imagenoscura2[i,j] = colorRGB2 # B,G,R

        im2 = Image.fromarray(imagenoscura2)
        imgtk2 = ImageTk.PhotoImage(image=im2)

        #img_label2 = Label(win, image= imgtk2,relief=SUNKEN,borderwidth=4).pack()
        #img_label2.place(x = 450,y = 120).pack()

        Label(mp, image= imgtk2,relief=SUNKEN,borderwidth=4,width=320, height=170).place(x = 425,y = 195).grid(row=0, column=0)
        canvas2.create_image(image=imgtk2)

    canvas2.place(x=410, y=180)

    botonimg1 = Button(mp, text = "Show color 2", width=25, command = colorimg2, fg='black',font='Verdana 8 bold',relief=RAISED ,borderwidth=4,justify=CENTER,cursor = 'hand2')
    botonimg1.place(x=490, y =130)


    ###### Calcular DELTA E

    def deltaE():
        L = float(Limg1.get())
        A = float(Aimg1.get())
        B = float(Bimg1.get())
        
        L2 = float(Limg2.get())
        A2 = float(Aimg2.get())
        B2 = float(Bimg2.get())


        
        difference = np.sqrt((L-L2)**2 + (A-A2)**2 + (B-B2)**2)
        if cv2.countNonZero(difference) == 0:
            deltae = (f'Colores iguales con un ΔE de: {np.amax(difference):.4f}')
            #print(f'Colores iguales ')
            label_deltaE = Label(mp, text=deltae, fg='black', font='Verdana 10 bold',relief=RAISED ,borderwidth=6,justify=CENTER)
            label_deltaE.place(x = 240, y = 410)
    
        else:
            deltae = (f'Colores diferentes con un ΔE de: {np.amax(difference):.3f}')
            #print(f'La diferencia entre los colores es de {np.amax(difference):.3f}')
            label_deltaE = Label(mp, text=deltae, bg = '#ff0000', fg='white', font='Verdana 10 bold',relief=RAISED ,borderwidth=6,justify=CENTER)
            label_deltaE.place(x = 240, y = 410)
        

            
    procesar = Button(mp, text = "PROCESAR", cursor = 'hand2', command = deltaE, fg='black',font='Verdana 8 bold',relief=RAISED ,borderwidth=4,justify=CENTER)
    procesar.place(x=350, y =130)


    def regresar():
        mp.withdraw()
        logging.deiconify()

    botton_regresar = Button(mp, text = 'Back',command = regresar, cursor = 'hand2', relief=RAISED,font =('Candara',10, 'bold'))
    botton_regresar.place(x=10,y=430)

    mp.mainloop()


## BOTONES DE CONTROL DE ACCESO 

# ENTRADAS TEXTOS USUARIO Y PASSWORD 
entrada_usuario_label = Label(logging, text = 'user',justify='center',relief=RIDGE,borderwidth=4,width=14,font =('Candara',15, 'bold'))
entrada_usuario_label.place(x=80,y=100)
entrada_usuario = Entry(logging, textvariable=usuario, width=25, relief=SUNKEN, borderwidth=4,justify='center')
entrada_usuario.place(x=80,y=140)
entrada_passwors_label = Label(logging, text = 'password' ,justify='center',relief=RIDGE,borderwidth=4, width=14,font =('Candara',15, 'bold'))
entrada_passwors_label.place(x=80,y=200)
entrada_passwors = Entry(logging, textvariable=password, show = '*' ,width=25, relief=SUNKEN, borderwidth=4,justify='center')
entrada_passwors.place(x=80,y=240)


# Crear botones
### Boton entrar
#Sombra
sombra = Label(logging, relief=RIDGE,borderwidth=4,width=92, height=5, bg='gray')
sombra.place(x=80,y=448)

entrar = Button(logging, text ='Module A',command=log ,cursor = 'hand2', bg= 'snow', width =20, relief = RAISED, font =('Candara',12, 'bold'),borderwidth=6)
entrar.place(x=110, y=470)

### Boton entrar modulo COLORANTES
salir = Button(logging, text ='Module C', command = materiaprima, cursor = 'hand2', bg= 'ghost white', width =20, relief = RAISED, font =('Candara',12, 'bold'),borderwidth=6)
salir.place(x=510, y=470)

### Boton salir
salir = Button(logging, text ='Clouse', command = irse, cursor = 'hand2', bg= 'ghost white', width =10, relief = RAISED, font =('Candara',12, 'bold'),borderwidth=6)
salir.place(x=650, y=320)

### BOTON ENTRAR A CALIDAD
salir = Button(logging, text ='Module B', command = c_calidad, cursor = 'hand2', bg= 'ghost white', width =20, relief = RAISED, font =('Candara',12, 'bold'),borderwidth=6)
salir.place(x=310, y=470)

### BOTON MOSTRAR DF ID
datos_id = Button(logging, text ='Data A', command = df_id, cursor = 'hand2', bg= 'ghost white', width =10, relief = RAISED, font =('Candara',10, 'bold'),borderwidth=6)
datos_id.place(x=670, y=50)

### BOTON MOSTRAR DF CALIDAD
datos_cc = Button(logging, text ='Data B', command = df_cc, cursor = 'hand2', bg= 'ghost white', width =10, relief = RAISED, font =('Candara',10, 'bold'),borderwidth=6)
datos_cc.place(x=670, y=90)


logging.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\juand\anaconda3\envs\creaapli\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\juand\AppData\Local\Temp/ipykernel_19860/1554257507.py", line 1007, in colorimg1
    Label(mp, image= imgtk,relief=SUNKEN,borderwidth=4,width=320, height=170).place(x = 25,y = 195).grid(row=0, column=0)
AttributeError: 'NoneType' object has no attribute 'grid'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\juand\anaconda3\envs\creaapli\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\juand\AppData\Local\Temp/ipykernel_19860/1554257507.py", line 1069, in colorimg2
    Label(mp, image= imgtk2,relief=SUNKEN,borderwidth=4,width=320, height=170).place(x = 425,y = 195).grid(row=0, column=0)
AttributeError: 'NoneType' object has no attribute 'grid'
Exception in Tkinter callback
Traceback (most recent call last):

## ACTUALIZANDO ETIQUETA PASA L A o B

In [37]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.
